## PROBANDO TFLITEAR MODELOS

In [1]:
import tensorflow as tf
import os

In [2]:
os.getcwd()

'/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [3]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model_functional_production_with_NMS/my_model')
converter.experimental_new_converter = True
converter.experimental_new_quantizer = True
converter.allow_custom_ops = True

In [4]:
current_dir = os.getcwd()

In [5]:
val_dir_path = "/tf/home/sergio/Tesis/COCODataset/pedestrian_dataset_val"

In [6]:
os.chdir(val_dir_path)
file_names = os.listdir()[:200]

In [7]:
from skimage.io import imread

In [8]:
def representative_data_gen():
    for i in file_names:
        img_raw = tf.image.decode_image(open(i, 'rb').read(), channels=3)
        img = tf.expand_dims(img_raw, 0)
        img = tf.image.resize(img, (416, 416))/255
        yield [img]

In [9]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
converter.representative_dataset = representative_data_gen
tflite_quant_model = converter.convert()

RuntimeError: Quantization not yet supported for op: 

In [ ]:
os.chdir(current_dir)
open("converted_model.tflite", "wb").write(tflite_quant_model)

# haciendo inferencia

In [23]:
interpreter_quant = tf.lite.Interpreter(model_path=current_dir+"/converted_model.tflite")
interpreter_quant.allocate_tensors()
input_index_quant = interpreter_quant.get_input_details()[0]["index"]
output_index_quant = interpreter_quant.get_output_details()[0]["index"]

In [24]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

In [25]:
image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = (tf.image.resize(img, (416, 416))).numpy()

In [26]:
img.shape

(1, 416, 416, 3)

In [27]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, img)
interpreter_quant.invoke()
predictions = interpreter_quant.get_tensor(output_index)

In [28]:
predictions.shape

(1, 2535, 5)

In [29]:
%timeit interpreter_quant.invoke()

44.3 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
import time
tiempo = []
with tf.device("CPU:0"):
    for i in range(100):
        inicio = time.time()
        interpreter_quant.invoke()
        fin = time.time()
        tiempo.append(fin-inicio)
print(tiempo)

[0.046181678771972656, 0.0446629524230957, 0.04484081268310547, 0.04472064971923828, 0.04509544372558594, 0.04464077949523926, 0.04487752914428711, 0.04489874839782715, 0.04487967491149902, 0.04416966438293457, 0.04425215721130371, 0.04438185691833496, 0.044146060943603516, 0.04434919357299805, 0.04417681694030762, 0.04414248466491699, 0.04418826103210449, 0.04417157173156738, 0.04422640800476074, 0.04423928260803223, 0.04424691200256348, 0.04432415962219238, 0.04493451118469238, 0.04498434066772461, 0.04469895362854004, 0.04487752914428711, 0.0449519157409668, 0.045465946197509766, 0.044605255126953125, 0.04449820518493652, 0.04428434371948242, 0.044173479080200195, 0.04525494575500488, 0.04429960250854492, 0.04416990280151367, 0.04420042037963867, 0.04421830177307129, 0.044228315353393555, 0.044168710708618164, 0.044252634048461914, 0.0439753532409668, 0.04446840286254883, 0.044260501861572266, 0.044052839279174805, 0.04415082931518555, 0.04414176940917969, 0.04416298866271973, 0.044

In [20]:
1/0.045

22.22222222222222